In [1]:
"""
Created on Tue Jun 18 08:58:21 2019
Euclidean Decoding Testing for RS code of length 7 over GF(64)=GF(2^6)
using p(x)=1+x+x^6
(63,47,17) RS code with t=8, d_min=2*t+1=17,
n=63, n-k=2*t=16, k=n-(n-k)=63-16=47
@author: Young Joon Song
"""


import numpy as np
from Source.GF64 import GFE
m=6

order_alpha=2**m-1
t_RS=8
length_parity=2*t_RS # n-k=2*8=16 symbols
b_RS=1          # primitive RS code
n_RS=order_alpha # n=63 symbols
k_RS=n_RS - length_parity # n=63, n-k=2*8=16, k=n-(n-k)=63-16=47 


Symbol_error_count=0
Z_RS=[]  # zeros of RS code
for i in range(2*t_RS):    Z_RS.append((b_RS+i)%order_alpha)
print('Zeros(RS):',Z_RS)

def GF_Index(cc):
    for GFindex, GFvector in GFE.items():
        if GFvector == list(cc): return GFindex
    print("Invalid vector of m bits")

def GF_Add(a,b): # alpha^c = alpha^a + alpha^b
    aa=GFE.get(a)
    bb=GFE.get(b)
    cc=np.add(aa,bb)%2
    return GF_Index(cc)

def GF_MUL(a,b): 
    if a >= 0 and b>=0: # if a and b are greater than -1, alpha^a * alpha^b =alpha^(c), c=a+b (mod n)
        c=(a+b)%order_alpha
        return c
    else:
        return -1  # if a and/or b is minus, the result is zero vector that is, alpha^-1

def GF_MUL_Inv(a): # alpha^a * alpha^b = alpha^0
    b=-1*a%order_alpha    
    return b

def POL_MUL(a_coeff,b_coeff):
    conv_window=len(b_coeff)
    sr_size=conv_window-1
    num_computation=len(a_coeff)+sr_size
    buff=-1*np.ones(conv_window) ## setting zero vectors
    c_coeff=[]
    input_coeff=list(a_coeff)
    for i in range(sr_size): input_coeff.append(-1)
    
#   print('input coeff:',input_coeff)
    
    for i in range(num_computation):
        for j in range(conv_window-2,-1,-1): buff[j+1]=buff[j]
        buff[0]=input_coeff[i]
#       print('Buffer:',buff)
        
        branch_mul=[]
        for j in range(conv_window):
            dum1=GF_MUL(b_coeff[j], buff[j])
            branch_mul.append(dum1)
#       print('Branch Multip:',branch_mul)
        
        branch_add=-1
        for j in range(len(branch_mul)):
            branch_add=GF_Add(branch_add, branch_mul[j])
        
#        print('Branch add:',branch_add)
        c_coeff.append(branch_add)
        
    return c_coeff


def Parity_RS(message_RS,generator_RS):
    sr_size=len(generator_RS)-1
    sr_content=-1*np.ones(sr_size)
    for i in range(len(message_RS)-1,-1,-1):
        feedback_add=GF_Add(sr_content[-1],message_RS[i])
        feedback=GF_MUL(feedback_add, generator_RS[-1])
#        print('feedback:',feedback)
        branch_output=[]
        for j in range(sr_size):
            dum2=GF_MUL(feedback, generator_RS[j])
            branch_output.append(dum2)
#        print('branch output:',branch_output) 
        for j in range(sr_size-1,0,-1):
            sr_content[j]=GF_Add(sr_content[j-1], branch_output[j])
        sr_content[0]=branch_output[0]
#        print('sr:',sr_content)
    return sr_content

def POL_DIV(Dividend,Divisor):
    Dividend_coeff=[]
    for i in range(len(Dividend)): Dividend_coeff.append(Dividend[i])
    Divisor_coeff=[]
    for i in range(len(Divisor)): Divisor_coeff.append(Divisor[i])
    Divisor_coeff[-1]=(-1*Divisor_coeff[-1])%(2**m-1)
#    print("Div coeff:",Divisor_coeff)
    Quotient_coeff=[]
    Remainder_coeff=[]
    sr_size=len(Divisor_coeff)-1
    sr_content=-1*np.ones(sr_size)
    for i in range(len(Dividend)-1,-1,-1):
        feedback=GF_MUL(sr_content[-1],Divisor_coeff[-1])
#        print('feedback:',feedback)
        Quotient_coeff.append(feedback)
        
        branch_output=[]
        for j in range(sr_size):
            dum2=GF_MUL(feedback, Divisor_coeff[j])
            branch_output.append(dum2)
#        print('branch output:',branch_output)
        for j in range(sr_size-1,0,-1):
            sr_content[j]=GF_Add(sr_content[j-1], branch_output[j])
        sr_content[0]=GF_Add(branch_output[0], Dividend_coeff[-1])
        del  Dividend_coeff[-1]
#        print('Shift reg content:',sr_content)
    for i in range(sr_size): Remainder_coeff.append(sr_content[i])
    for i in range(sr_size):
        if Remainder_coeff[-1] < 0: 
            del Remainder_coeff[-1]
        else: break
            
    Quotient_coeff.reverse()
    for i in range(len(Quotient_coeff)):
        if Quotient_coeff[-1] < 0:
            del Quotient_coeff[-1]
        else: break
    
    return (Quotient_coeff,Remainder_coeff) # sr_content = coefficients of remainder #

def POL_Add(a_coeff, b_coeff):
    max_a_b_length=max(len(a_coeff),len(b_coeff))
    min_a_b_length=min(len(a_coeff),len(b_coeff))
    a_b_GF_addition_coeff=[]
    for i in range(min_a_b_length):
        dum=GF_Add(a_coeff[i],b_coeff[i])
        a_b_GF_addition_coeff.append(dum)
    if len(a_coeff) == len(b_coeff): return a_b_GF_addition_coeff
    if len(a_coeff) > len(b_coeff):
        for i in range(min_a_b_length,max_a_b_length):
            a_b_GF_addition_coeff.append(a_coeff[i])
        return a_b_GF_addition_coeff
    else:
        for i in range(min_a_b_length,max_a_b_length):
            a_b_GF_addition_coeff.append(b_coeff[i])
        return a_b_GF_addition_coeff
 
# GFE_Index*j mod Order_GFE, j=0,1,..,Vector_length-1 #          
def GFE_Power_vector(GFE_Index,Order_GFE,Vector_length):
    vector_list=[]
    if GFE_Index < 0: 
        for i in range(Vector_length): vector_list.append(-1)
    else:
        for i in range(Vector_length): vector_list.append((GFE_Index*i)%Order_GFE)
    return vector_list


# Innder Product of Two GF Vectors #
def GF_InnerProduct(A_GF_Vector,B_GF_Vector):
    Vector_Length=min(len(A_GF_Vector),len(A_GF_Vector))
    GF_Sum=-1
    GF_Product_Vector=[]
    for i in range(Vector_Length):
        GF_Product_Vector.append(GF_MUL(A_GF_Vector[i],B_GF_Vector[i]))
    for i in range(Vector_Length):
        GF_Sum=GF_Add(GF_Sum,GF_Product_Vector[i])
    return GF_Sum

def GF_Function_Value(GFE_Index,Order_GFE,GF_Function_coeff):
    A_GF_Vector=GFE_Power_vector(GFE_Index,Order_GFE,len(GF_Function_coeff))
    Function_Value=GF_InnerProduct(A_GF_Vector,GF_Function_coeff)
    return Function_Value
    
   
# From a0+a1*x+a2*x^2+... ==> To: a1+a2*2*x+... #
def GF_Derivative(GFVector):
    GFVector_Prime=[]
    A_GF_Vector=[]
    for i in range(len(GFVector)):
        if i%2 == 0:
            A_GF_Vector.append(-1)
        else:
            A_GF_Vector.append(0)
    for i in range(len(GFVector)):
        GFVector_Prime.append(GF_MUL(A_GF_Vector[i],GFVector[i]))
    
    del GFVector_Prime[0]
    return(GFVector_Prime)

Zeros(RS): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


In [2]:
def RS_Decoding(r_vector_RS, syndrome_check, syndrome_RS):
    if syndrome_check == 0:
    # Extract message #
        Estimated_Message=r_vector_RS[-1*len(msg_RS)::]
    else:
        A_coeff=[]
        for i in range(2*t_RS): A_coeff.append(-1) 
        A_coeff.append(0)

        B_coeff=list(syndrome_RS)

        REMAINDER=[]
        REMAINDER.append(A_coeff)
        REMAINDER.append(B_coeff)
    
        T_coefficient=[]
        T_coefficient.append([-1])  ## alpha^-1 = 0 ##
        T_coefficient.append([0])  ## alpha^0 = 1 ##
    
    
        QUOTIENT=[]
        cnt_i=0
        while len(REMAINDER[-1])-1 >= t_RS:
            QUO, REM = POL_DIV(REMAINDER[-2],REMAINDER[-1])
            QUOTIENT.append(QUO)
            REMAINDER.append(REM)
            Dummy_A=T_coefficient[-2]
            Dummy_B=POL_MUL(QUOTIENT[-1],T_coefficient[-1])
            Dummy_C=POL_Add(Dummy_A,Dummy_B)
            T_coefficient.append(Dummy_C)
            cnt_i+=1

        Kappa=(-1*T_coefficient[-1][0])%order_alpha 
        Error_Location_coeff=[]
        for i in range(len(T_coefficient[-1])):
            Dummy=T_coefficient[-1][i]
            Error_Location_coeff.append(GF_MUL(Kappa,Dummy))
    
        for i in range(len(T_coefficient[-1])):
            if T_coefficient[-1][-1] < 0:
                del T_coefficient[-1][-1]
            else: break
    
        Error_Evaluation_coeff=[]
        for i in range(len(REMAINDER[-1])):
            Dummy=REMAINDER[-1][i]
            Error_Evaluation_coeff.append(GF_MUL(Kappa,Dummy))

        error_positions=[]
        Chien_Search_Values=[]
        num_errors=0
        for i in range(order_alpha):
            alpha_minus_i=(-1*i)%order_alpha ## alpha^(-i mod order_alpha), i=0,1,...,order_alpha ##
            Error_Location_Function_Value=GF_Function_Value(alpha_minus_i,order_alpha,Error_Location_coeff)
            Chien_Search_Values.append(Error_Location_Function_Value)
            if Error_Location_Function_Value == -1: 
                error_positions.append(i)
                num_errors+=1

        Error_Location_Der=GF_Derivative(Error_Location_coeff)
    
        Error_Numerator=[]
        Error_Denominator=[]
        Error_Estimated_Values=[]
        Estimated_Error_Vector=-1*np.ones(len(r_vector_RS)) # Error Vector #
        for i in range(num_errors):
            Inv_Xi_Index=(-1*error_positions[i])%order_alpha
            Numerator_i=GF_Function_Value(Inv_Xi_Index,order_alpha,Error_Evaluation_coeff)
            Error_Numerator.append(Numerator_i)
            Denominator_i=GF_Function_Value(Inv_Xi_Index,order_alpha,Error_Location_Der)
            Error_Denominator.append(Denominator_i)
            Estim_Error_Value_i=GF_MUL(Numerator_i,GF_MUL_Inv(Denominator_i))
            Error_Estimated_Values.append(Estim_Error_Value_i)
            Estimated_Error_Vector[error_positions[i]]=Estim_Error_Value_i
    
        Estimated_Codeword=POL_Add(r_vector_RS, Estimated_Error_Vector)
    

        Estimated_Message=Estimated_Codeword[-1*len(msg_RS)::]
    
    return Estimated_Message

In [3]:
##### Generation polynomial : g(x)  #####
a_x=[0]
a_x.append(Z_RS[0])
b_x=[0]
for i in range(1,len(Z_RS)):
    b_x.append(Z_RS[i])
#    print('a(x)=',a_x)
#    print('b(x)=',b_x)
    c_x=POL_MUL(a_x,b_x)  ## c(x)=a(x)b(x)
    a_x=c_x
    b_x=[0]

print('c(x):',c_x)
g_x=c_x  
g_x.reverse()   ## g(x)=g_0+g_1*x+...+g_n-k*x^n-1
print('g(x)=g_0+g_1*x+...+g_n-k*x^n-1: ',g_x)
print('length of g:',len(g_x))





c(x): [0, 28, 41, 1, 16, 24, 54, 33, 38, 50, 25, 12, 21, 23, 17, 21, 10]
g(x)=g_0+g_1*x+...+g_n-k*x^n-1:  [10, 21, 17, 23, 21, 12, 25, 50, 38, 33, 54, 24, 16, 1, 41, 28, 0]
length of g: 17


In [4]:
from keras.models import load_model
model = load_model('model/model_p6_sigmoid.h5')

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [5]:
##### RS encoding #########
np.random.seed(700)
r_vector_RS=[]
msg_RS=[]
error_RS=[]
SER=[]
ad_SER=[]
non_decoding_ad_SER=[]
for i in range(k_RS):
    msg_RS.append(np.random.randint(-1,order_alpha))
parity_symbols=Parity_RS(msg_RS,g_x)
codeword_RS=list(parity_symbols)+list(msg_RS)

SNR=np.arange(1,11)  # in dB
Max_Decoding_errors=500
for SNR_count, SNR_real in enumerate(SNR):
    print(SNR_real)
    Sigma=np.sqrt(1/(2*SNR_real))
    Total_Symbol_error = 0
    Total_Word_error=0
    Total_ad_Symbol_error = 0
    Total_ad_Word_error=0
    Total_non_decoding_ad_Symbol_error =0
    Max_Number_Codewords=0
    r_error_count =0
    ad_error_count =0
    while(1):
        Max_Number_Codewords+=1
        r_vector_RS=[]
        ad_vector_RS=[]
        for i in range(n_RS):
            m_bits=GFE.get(codeword_RS[i])
            tx=[]
            rx=[]
            rx_Hard=[]
            rn=np.random.normal(loc=0.0,scale=Sigma,size=m)
            for j in range(m): tx.append(np.power(-1,m_bits[j]))
            for j in range(m):
                Bi_AWGN=tx[j]+rn[j]
                rx.append(Bi_AWGN)
                if Bi_AWGN >=0 :
                    rx_Hard.append(0)
                else:
                    rx_Hard.append(1)

            r_vector_RS.append(GF_Index(rx_Hard))
            model_result = model.predict(np.reshape(rx, (1,m)))
            ad_RS = GF_Index(np.where(model_result>0.5,1,0)[0])
            ad_vector_RS.append(ad_RS)

        for i in range(n_RS):
            error_RS.append(GF_Add(codeword_RS[i], r_vector_RS[i]))
            if int(r_vector_RS[i] != int(codeword_RS[i])):
                r_error_count+=1
        
        
        
        for i in range(n_RS):
            error_RS.append(GF_Add(codeword_RS[i], ad_vector_RS[i]))
            if int(ad_vector_RS[i] != int(codeword_RS[i])):
                ad_error_count+=1 
        
    ### syndrome check ###
        syndrome_RS=-1*np.ones(2*t_RS)
        for i in range(2*t_RS):
            for j in range(len(codeword_RS)):
                dum1=GF_MUL(r_vector_RS[j], Z_RS[i]*j)
                syndrome_RS[i]=GF_Add(dum1, syndrome_RS[i])

        syndrome_check=0
        for i in range(2*t_RS):
            if syndrome_RS[i] >= 0:
                syndrome_check+=1
        
        ad_syndrome_RS = -1*np.ones(2*t_RS)
        for i in range(2*t_RS):
            for j in range(len(codeword_RS)):
                dum1=GF_MUL(ad_vector_RS[j], Z_RS[i]*j)
                ad_syndrome_RS[i]=GF_Add(dum1, ad_syndrome_RS[i])
                
        ad_syndrome_check=0
        for i in range(2*t_RS):
            if ad_syndrome_RS[i] >= 0:
                ad_syndrome_check+=1
    
        Estimated_Message = RS_Decoding(r_vector_RS, syndrome_check, syndrome_RS)
        ad_Estimated_Message = RS_Decoding(ad_vector_RS, ad_syndrome_check, ad_syndrome_RS)
    
    # SER Calculation #
        Symbol_error_count=0
        for i in range(len(msg_RS)):
            if Estimated_Message[i] != msg_RS[i]:
                Symbol_error_count+=1
        
        if Symbol_error_count !=0:
            Total_Word_error+=1
            Total_Symbol_error += Symbol_error_count
        
        ad_Symbol_error_count=0
        for i in range(len(msg_RS)):
            if ad_Estimated_Message[i] != msg_RS[i]:
                ad_Symbol_error_count+=1
        
        if ad_Symbol_error_count !=0:
            Total_ad_Word_error+=1
            Total_ad_Symbol_error += ad_Symbol_error_count
        
        
        if Total_Word_error >=Max_Decoding_errors and Total_ad_Word_error >= Max_Decoding_errors:
            break
        if Max_Number_Codewords >= 5*10**3:
            break
            
    print('Total Tx codeword:', Max_Number_Codewords)
    SER_value=Total_Symbol_error/(len(msg_RS)*Max_Number_Codewords)
    ad_SER_value=Total_ad_Symbol_error/(len(msg_RS)*Max_Number_Codewords)
    SER.append(SER_value)
    ad_SER.append(ad_SER_value)
    print('rs',  SER[SNR_count])
    print('ad', ad_SER[SNR_count])
    print('r vector', r_error_count/(n_RS*Max_Number_Codewords))
    print('non_decoding_ad', ad_error_count/(n_RS*Max_Number_Codewords))
    
print(SNR)
print(SER)
print(ad_SER)


1
Total Tx codeword: 500
rs 0.3975744680851064
ad 0.39863829787234045
r vector 0.388984126984127
non_decoding_ad 0.3914920634920635
2
Total Tx codeword: 1158
rs 0.07705875868151252
ad 0.08190938154558483
r vector 0.1281492447295556
non_decoding_ad 0.13130191627600954
3


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-3e8db02b2b0b>", line 47, in <module>
    model_result = model.predict(np.reshape(rx, (1,m)))
  File "D:\anaconda\lib\site-packages\keras\engine\training.py", line 1462, in predict
    callbacks=callbacks)
  File "D:\anaconda\lib\site-packages\keras\engine\training_arrays.py", line 324, in predict_loop
    batch_outs = f(ins_batch)
  File "D:\anaconda\lib\site-packages\tensorflow_core\python\keras\backend.py", line 3476, in __call__
    run_metadata=self.run_metadata)
  File "D:\anaconda\lib\site-packages\tensorflow_core\python\client\session.py", line 1472, in __call__
    run_metadata_ptr)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\anaconda\lib\site-packages\IPython\core\interactiveshell.p

KeyboardInterrupt: 